In [ ]:
#Lấy data unique
import pandas as pd
df = pd.read_csv('data.csv')
df_unique = df.drop_duplicates(subset='url')
df_unique['id'] = range(1, len(df_unique) + 1)
# Đưa cột 'id' lên đầu
columns_order = ['id'] + [col for col in df_unique.columns if col != 'id']
df_unique = df_unique[columns_order]
# Lưu DataFrame vào file CSV
df_unique.to_csv('dataUnique.csv', index=False)

In [ ]:
#Chuyển thể loại thành id
import pandas as pd
df = pd.read_csv('dataUnique.csv')

# Danh sách đối tượng genres
genre_list = [
    {"id": 1, "name": "Phiêu Lưu - Hành Động"},
    {"id": 2, "name": "Kinh Dị - Ma"},
    {"id": 3, "name": "Võ Thuật - Kiếm Hiệp"},
    {"id": 4, "name": "Hình Sự - Chiến Tranh"},
    {"id": 5, "name": "Khoa Học - Viễn Tưởng"},
    {"id": 6, "name": "Phim Hoạt Hình"},
    {"id": 7, "name": "Tài Liệu - Khám Phá"},
    {"id": 8, "name": "Hài Hước"},
    {"id": 9, "name": "Gia Đình - Học Đường"},
    {"id": 10, "name": "Thể Thao - Âm Nhạc"},
    {"id": 11, "name": "Tâm Lý - Tình Cảm"},
    {"id": 12, "name": "Boy Love"},
    {"id": 13, "name": "Cổ Trang - Thần Thoại"},
    {"id": 14, "name": "Bí Ẩn - Siêu Nhiên"},
    {"id": 15, "name": "TV Show"}
]

# Tạo một từ điển ánh xạ từ name sang id
genre_mapping = {genre['name']: genre['id'] for genre in genre_list}
def get_genre_ids(genre_names):
    genre_ids = []
    for name in genre_names.split(','):
        name = name.strip()  # Loại bỏ khoảng trắng
        genre_id = genre_mapping.get(name, None)  # Trả về None nếu không tìm thấy
        if genre_id is not None:
            genre_ids.append(str(genre_id))  # Thêm id vào danh sách
    return ','.join(genre_ids)

df['genresId'] = df['genres'].apply(get_genre_ids)

df.to_csv('data_1.csv', index=False)



In [ ]:
#chuyển quốc gia thành id
import pandas as pd
df = pd.read_csv('data_1.csv')
country_mapping = {
    "Nhật Bản": 1,
    "Đài Loan": 2,
    "Âu - Mỹ": 3,
    "Thái Lan": 4,
    "Hồng Kông": 5,
    "Canada": 6,
    "Hàn Quốc": 7,
    "Quốc Gia Khác": 8,
    "Trung Quốc": 9
}
# Hàm ánh xạ từ tên quốc gia sang id
def get_country_id(name):
    return country_mapping.get(name, None)
df['CountryId'] = df['CountryId'].apply(get_country_id)
df.to_csv('data_2.csv', index=False)

In [ ]:
#fix releaseDate
import pandas as pd
import random
from datetime import datetime, timedelta

# Đọc dữ liệu
df = pd.read_csv('data_2.csv')

# Tính ngày giới hạn (hiện tại + 3 tháng)
today = datetime.today()
limit_date = today + timedelta(days=90)  # Cộng thêm 3 tháng (khoảng 90 ngày)

# Hàm tạo ngày ngẫu nhiên không vượt quá ngày giới hạn
def random_date_from_year_limited(year):
    if pd.notnull(year):  # Kiểm tra nếu year không rỗng
        # Tạo ngày đầu năm và cuối năm
        start_date = datetime(year, 1, 1)
        end_date = datetime(year, 12, 31)
        
        # Giới hạn ngày cuối cùng trong khoảng cho phép
        valid_end_date = min(end_date, limit_date)
        
        # Tạo ngày ngẫu nhiên trong khoảng hợp lệ
        random_date = start_date + timedelta(days=random.randint(0, (valid_end_date - start_date).days))
        return random_date.strftime("%Y-%m-%d")  # Định dạng thành chuỗi YYYY-MM-DD
    return None

# Áp dụng hàm vào cột year
df['releaseDate'] = df['year'].apply(lambda x: random_date_from_year_limited(int(x)))

# Lưu DataFrame mới ra file CSV
df.to_csv('data_3.csv', index=False)


In [ ]:
# xử lí số tập
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('data_3.csv')

# Xử lý cột "x Tập"
def extract_episode(value):
    if pd.notnull(value):  # Kiểm tra nếu không rỗng
        # Tách số từ chuỗi (lấy phần trước "Tập")
        num = ''.join(filter(str.isdigit, str(value)))
        return int(num) if num else 1  # Nếu có số thì trả về, nếu không trả về 1
    return 1  # Giá trị rỗng thì trả về 1

# Áp dụng hàm vào cột "x Tập"
df['TotalEpisodes'] = df['TotalEpisodes'].apply(extract_episode)

# Lưu DataFrame mới ra file CSV
df.to_csv('data_4.csv', index=False)


In [ ]:
#tạo data movies
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('data_4.csv')
df['Overview'] = df['Overview'].astype(str)

# Tạo danh sách chứa các lệnh SQL
sql_statements = []

# Duyệt từng hàng trong DataFrame để tạo câu lệnh INSERT
for index, row in df.iterrows():
    # Xác định Type
    Type = 'MOVIE' if row['TotalEpisodes'] == 1 else 'SERIES'
    
    # Tạo câu lệnh SQL
    sql = f"""
    INSERT INTO Movies (
        id, name, PosterPath, releaseDate, CountryId, Language, Overview, Type, TotalEpisodes
    ) VALUES (
        {row['id']}, 
        '{row['name'].replace("'", "''")}', 
        '{row['PosterPath'].replace("'", "''")}', 
        '{row['releaseDate']}', 
        {row['CountryId']}, 
        '{row['Language'].replace("'", "''")}', 
        '{row['Overview'].replace("'", "''")}', 
        '{Type}',
        {row['TotalEpisodes']}
    );
    """
    # Thêm câu lệnh vào danh sách
    sql_statements.append(sql.strip())

# Lưu các câu lệnh SQL vào file
with open('insert_movies.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_statements))

print("File SQL đã được tạo: insert_movies.sql")


In [ ]:
#sql moviegenre
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('data_4.csv')

# Tạo danh sách chứa các lệnh SQL
sql_statements = []

# Duyệt từng hàng trong DataFrame để tạo câu lệnh INSERT
for index, row in df.iterrows():
    gen = row['genresId'].split(',')
    for idgen in gen :
        sql = f"""
        INSERT INTO moviegenres (
             MovieId,GenreId
        ) VALUES (
            {row['id']},
            {idgen} 
        );
        """
        sql_statements.append(sql.strip())

# Lưu các câu lệnh SQL vào file
with open('insert_genres_movie.sql', 'w', encoding='utf-8') as file:
    file.write('\n'.join(sql_statements))



In [ ]:
#Tạo data user
from faker import Faker

# Khởi tạo Faker với locale Việt Nam
faker = Faker("vi_VN")

# Số lượng người dùng cần tạo
num_users = 200

# Khởi tạo danh sách lưu các câu lệnh SQL
sql_statements = []

# Tạo câu lệnh SQL cho từng người dùng
for i in range(1, num_users + 1):
    name = faker.name()  # Tên người Việt
    email = faker.email()  # Email
    password = faker.password(length=10)  # Mật khẩu giả định
    is_active = 1

    # Câu lệnh SQL dạng INSERT
    sql = f"INSERT INTO Users (Id, Name, Email, Password, IsActive) VALUES ({i}, '{name}', '{email}', '{password}', {is_active});"
    sql_statements.append(sql)

# Ghi các câu lệnh SQL ra file hoặc in ra màn hình
output_file = "insert_users_vietnamese.sql"
with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(sql_statements))

print(f"Generated {num_users} SQL INSERT statements with Vietnamese names. Output saved to {output_file}")


In [ ]:
#sql rating
import random

# Các tham số cơ bản
user_ids = list(range(1, 52))  # UserId từ 1 đến 51
movie_ids = list(range(1, 244))  # MovieId từ 1 đến 243
num_ratings = 400  # Số lượng dòng cần tạo

# Tập hợp để đảm bảo tính duy nhất của cặp (UserId, MovieId)
unique_pairs = set()

# Danh sách lưu các câu lệnh SQL
sql_statements = []

# Tạo dữ liệu mẫu
while len(unique_pairs) < num_ratings:
    user_id = random.choice(user_ids)
    movie_id = random.choice(movie_ids)
    
    # Nếu cặp này chưa tồn tại, thêm vào
    if (user_id, movie_id) not in unique_pairs:
        unique_pairs.add((user_id, movie_id))
        rating = random.randint(1, 5)  # Rating ngẫu nhiên từ 1 đến 5
        # Tạo câu lệnh SQL
        sql = f"INSERT INTO rating (UserId, MovieId, Rating) VALUES ({user_id}, {movie_id}, {rating});"
        sql_statements.append(sql)

# Ghi kết quả ra file hoặc in ra màn hình
output_file = "insert_ratings.sql"
with open(output_file, "w", encoding="utf-8") as file:
    file.write("\n".join(sql_statements))

print(f"Generated {num_ratings} SQL INSERT statements. Output saved to {output_file}")
